<h3> You will need to run 'pip install pypdf' </h3>

In [2]:
import os
from dotenv import load_dotenv
import glob
import gradio as gr
# Langchain imports
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings import OllamaEmbeddings, OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain_openai import ChatOpenAI

In [3]:
def selectModel(model, model_name = None):
    """
        This function allows you to select one of the models. The options are below.
            • 'Ollama'
            • 'OpenAI': requires an API key in the .env environment under the name of 'OPENAI_API_KEY'
        This function returns two elements: the LLM and embeddings used to vectorize the documents.
    """
    if model == 'Ollama':
        try:
            llm = ChatOllama(model=model_name)
            embeddings = OllamaEmbeddings(model=model_name)
        except Exception as error:
            raise Exception(f"Please ensure that your Ollama model name is correct. It current value is {model}. " + error)
    elif model == 'OpenAI':
        try:
            load_dotenv(override=True)
            os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'Your API key if it is not loaded in the .env file.')
        except Exception as error:
            raise EnvironmentError(f"Please ensure that the OpenAI API key has been inserted in the .env file under the name of 'OPENAI_API_KEY'. " + error)
        try:
            llm = ChatOpenAI(model="gpt-4o-mini")
            embeddings = OpenAIEmbeddings()
        except Exception as error:
            raise Exception(f"Please ensure that you have correctly imported the ChatOpenAI and OpenAIEmbeddings dependencies. " + error)
    
    return llm, embeddings

In [4]:
def createRAGVectorStore(embeddings, vector_db_name, documents):
    """
        This function will create the vector store for a RAG model.
        You need to pass the embeddings of the model that you will use, along with the name of the vector database.
    """
    if os.path.exists(vector_db_name):
        Chroma(persist_directory=vector_db_name, embedding_function=embeddings).delete_collection()
    
    try:
        return Chroma.from_documents(documents=documents, embedding=embeddings, persist_directory=vector_db_name)
    except Exception as e:
        raise Exception("Please verify that the documents and embeddings have been passed correctly.")

In [5]:
def createPDFDocumentsFromDirectory(folder):
    """
        This function creates the documents from the PDFs that will be used for the vector datastore.
        It returns a list of documents.
    """
    documents = []

    try:
        loader = DirectoryLoader(folder, glob="**/*.pdf", loader_cls=PyPDFLoader)
    except Exception as error:
        raise Exception(f"Please ensure that you have imported the DirectoryLoader and PyPDFLoader dependencies. " + error)

    documents = loader.load()

    for document in documents:
        # Add the name of the contract
        document.metadata['contract'] = document.metadata['source'].split('/')[1].split('.')[0]
    
    return documents

In [6]:
def createChunksFromDocuments(documents, size = 300, overlap = 200):
    try: 
        text_splitter = CharacterTextSplitter(chunk_size = size, chunk_overlap = overlap)
        chunks = text_splitter.split_documents(documents)
    except Exception as error:
        raise Exception("Please ensure that the CharacterTextSplitter dependency has been imported. " + error)

    return chunks

In [7]:
def setUpRAGModel(llm, vector_data_store):
    try:
        memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    except Exception as error:
        raise Exception(f"Please ensure that you have imported the ConversationBufferMemory function. " + error)
    
    try:
        retriever = vector_data_store.as_retriever()
    except Exception as error:
        raise TypeError("Please ensure that you are passing a vector datastore. Error: " + error)
    
    try:
        return ConversationalRetrievalChain.from_llm(llm=llm, memory=memory, retriever=retriever)
    except Exception as error:
        raise ImportError("Please ensure that you have imported the ConversationalRetrievalChain class. Error: " + error)

In [17]:
# Declare variables here
vector_db_name = 'contracts_db'
model = 'Ollama'
# If you are not planning on using Ollama, the functions will ignore the variable below.
model_name = 'llama3.2:1b'

In [18]:
llm, embeddings = selectModel(model, model_name)

# Test
llm.invoke("What do you think of the RAG architecture?").content

'I\'m not familiar with a specific architectural style or design approach called "RAG." It\'s possible that it\'s a lesser-known or emerging concept, or it could be a misnomer for an existing framework.\n\nCould you provide more context or information about what RAG is and what it refers to? I\'d be happy to try and help you understand the concept better.'

In [19]:
# Create documents
documents = createPDFDocumentsFromDirectory('Contracts')

# Chunk the documents for the vector data store
chunks = createChunksFromDocuments(documents)

# Create the vector data store
vector_store = createRAGVectorStore(embeddings=embeddings, vector_db_name=vector_db_name, documents=chunks)

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 10 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 24 0 (offset 0)
Ignoring wrong pointing object 35 0 (offset 0)
Ignoring wrong pointing object 40 0 (offset 0)
Ignoring wrong pointing object 52 0 (offset 0)
Ignoring wrong pointing object 54 0 (offset 0)


In [20]:
# Initiate the RAG-assisted LLM
chain = setUpRAGModel(llm, vector_data_store=vector_store)

In [21]:
chain.invoke({"question": "When does the Sailpoint agreement expire?"})['answer']

'The SailPoint Confidential SaaS Agreement ( Exhibit B) does not explicitly state when it expires. However, based on the context and the fact that SailPoint offers various subscription terms for its services, it is likely that the agreement has a renewal term.\n\nAccording to the Schedule provided with the document, the Subscription Term is for 12 months, but it also mentions that it may be renewed for successive 12-month periods unless either party delivers written notice of non-renewal at least 30 days prior to the expiration of the then-current Subscription Term.'

In [22]:
# Gradio function
def chat(question, history):
    response = chain.invoke({"question": question})
    return response['answer']

In [23]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
